<a href="https://colab.research.google.com/github/jlopetegui98/NER-ClinicalTrials-Elegibility-Criteria/blob/main/Roberta%2BLLM/evaluate_roberta_chia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# uncomment if working in colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# uncomment if using colab
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install seqeval
!pip install -q -U evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification,  Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
import evaluate
import torch

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
# dict for the entities (entity to int value)
simple_ent = {"Condition", "Value", "Drug", "Procedure", "Measurement", "Temporal", "Observation", "Person", "Device"}
sel_ent = {
    "O": 0,
    "B-Condition": 1,
    "I-Condition": 2,
    "B-Value": 3,
    "I-Value": 4,
    "B-Drug": 5,
    "I-Drug": 6,
    "B-Procedure": 7,
    "I-Procedure": 8,
    "B-Measurement": 9,
    "I-Measurement": 10,
    "B-Temporal": 11,
    "I-Temporal": 12,
    "B-Observation": 13,
    "I-Observation": 14,
    "B-Person": 15,
    "I-Person": 16,
    "B-Device": 17,
    "I-Device": 18
}

entities_list = list(sel_ent.keys())
sel_ent_inv = {v: k for k, v in sel_ent.items()}

In [19]:
root = '..'
root = './drive/MyDrive/TER-LISN-2024'
data_path = f'{root}/data'
models_path = f'{root}/models'

In [20]:
model_name = "roberta-base"

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [22]:
# tokenize and align the labels in the dataset
def tokenize_and_align_labels(sentence, tokenizer, flag = 'I'):
    """
    Tokenize the sentence and align the labels
    inputs:
        sentence: dict, the sentence from the dataset
        flag: str, the flag to indicate how to deal with the labels for subwords
            - 'I': use the label of the first subword for all subwords but as intermediate (I-ENT)
            - 'B': use the label of the first subword for all subwords as beginning (B-ENT)
            - None: use -100 for subwords
    outputs:
        tokenized_sentence: dict, the tokenized sentence now with a field for the labels
    """
    tokenized_sentence = tokenizer(sentence['tokens'], is_split_into_words=True, truncation=True, padding='max_length', max_length=512)

    labels = []
    all_word_ids = []
    for i, labels_s in enumerate(sentence['ner_tags']):
        word_ids = tokenized_sentence.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # if the word_idx is None, assign -100
            if word_idx is None:
                label_ids.append(-100)
            # if it is a new word, assign the corresponding label
            elif word_idx != previous_word_idx:
                label_ids.append(labels_s[word_idx])
            # if it is the same word, check the flag to assign
            else:
                if flag == 'I':
                    if entities_list[labels_s[word_idx]].startswith('I'):
                      label_ids.append(labels_s[word_idx])
                    else:
                      label_ids.append(labels_s[word_idx] + 1)
                elif flag == 'B':
                    label_ids.append(labels_s[word_idx])
                elif flag == None:
                    label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        all_word_ids.append(word_ids)
    tokenized_sentence['labels'] = labels
    tokenized_sentence['word_ids'] = all_word_ids
    return tokenized_sentence

In [23]:
dataset = load_dataset('JavierLopetegui/chia_v1')

Generating train split:   0%|          | 0/8881 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1307 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/2221 [00:00<?, ? examples/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index'],
        num_rows: 8881
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index'],
        num_rows: 1307
    })
    val: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index'],
        num_rows: 2221
    })
})

In [25]:
# tokenize and align the labels in the dataset
dataset = dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, 'I'), batched = True)

Map:   0%|          | 0/8881 [00:00<?, ? examples/s]

Map:   0%|          | 0/1307 [00:00<?, ? examples/s]

Map:   0%|          | 0/2221 [00:00<?, ? examples/s]

In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 8881
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 1307
    })
    val: Dataset({
        features: ['tokens', 'ner_tags', 'file', 'index', 'input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 2221
    })
})

In [27]:
# load model
model = torch.load(f'{models_path}/roberta-ner-chia.pt')

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
data_for_model = dataset['test'].remove_columns(['file', 'tokens', 'labels', 'index', 'ner_tags', 'word_ids'])

In [30]:
data_for_model

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1307
})

In [31]:
data_loader = torch.utils.data.DataLoader(data_for_model, batch_size=8)

In [32]:
model.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [33]:
from tqdm import tqdm

In [34]:
len(data_loader.dataset[2]['attention_mask'])

512

In [35]:
labels = []
for batch in tqdm(data_loader):

    batch['input_ids'] = torch.LongTensor(np.column_stack(np.array(batch['input_ids']))).to(device)
    batch['attention_mask'] = torch.LongTensor(np.column_stack(np.array(batch['attention_mask']))).to(device)
    batch_tokenizer = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}
    # break
    with torch.no_grad():
        outputs = model(**batch_tokenizer)

    labels_batch = torch.argmax(outputs.logits, dim=2).to('cpu').numpy()
    labels.extend([list(labels_batch[i]) for i in range(labels_batch.shape[0])])

    del batch
    del outputs
    torch.cuda.empty_cache()

100%|██████████| 164/164 [00:43<00:00,  3.76it/s]


In [36]:
def annotate_sentences(dataset, labels, entities_list,criteria = 'first_label'):
    """
    Annotate the sentences with the predicted labels
    inputs:
        dataset: dataset, dataset with the sentences
        labels: list, list of labels
        entities_list: list, list of entities
        criteria: str, criteria to use to select the label when the words pices have different labels
            - first_label: select the first label
            - majority: select the label with the majority
    outputs:
        annotated_sentences: list, list of annotated sentences
    """
    annotated_sentences = []
    for i in range(len(dataset)):
        # get just the tokens different from None
        sentence = dataset[i]
        word_ids = sentence['word_ids']
        sentence_labels = labels[i]
        annotated_sentence = [[] for _ in range(len(dataset[i]['tokens']))]
        for word_id, label in zip(word_ids, sentence_labels):
            if word_id is not None:
                annotated_sentence[word_id].append(label)
        annotated_sentence_filtered = []
        if criteria == 'first_label':
            annotated_sentence_filtered = [annotated_sentence[i][0] for i in range(len(annotated_sentence))]
        elif criteria == 'majority':
            annotated_sentence_filtered = []
            for j in range(len(annotated_sentence)):
                starts_flag = entities_list[annotated_sentence[j][0]].startswith('B')

                ent = max(set(annotated_sentence[j]), key=annotated_sentence[j].count)
                if starts_flag and ent != 0:
                    label = entities_list[ent][2:]
                    label = 'B-' + label
                    annotated_sentence_filtered.append(sel_ent[label])
                else:
                    annotated_sentence_filtered.append(ent)
        annotated_sentences.append(annotated_sentence_filtered)
    return annotated_sentences

In [37]:
annotated_sentences_first = annotate_sentences(dataset['test'], labels, entities_list, criteria='first_label')
annotated_sentences_max = annotate_sentences(dataset['test'], labels, entities_list, criteria='majority')

In [38]:
#load seqeval metric for evaluation
metric = load_metric("seqeval")

<ipython-input-38-653dc96d1cff>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [39]:
def compute_metrics_tr(p):
    """
    Compute the metrics for the model
    inputs:
        p: tuple, the predictions and the labels
    outputs:
        dict: the metrics
    """
    predictions, labels = p

    # Remove ignored index (special tokens)
    true_predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    resutls_strict = metric.compute(predictions=true_predictions, references=true_labels, mode='strict', scheme='IOB2')

    cr1 = classification_report(true_labels, true_predictions)
    cr2 = classification_report(true_labels, true_predictions, mode='strict', scheme=IOB2)

    return results, resutls_strict,cr1,cr2

In [40]:
def get_full_entities(sent_labels):
    """
    Given a list of sentence annotations return a list of tuples with entitie and start and end index
    inputs:
        sent_labels: list of labels for an individual sentence
    output:
        entities: list of tuples (ent-type, start_index, end_index)
    """


In [41]:
def get_labels(p):
    predictions, labels = p
    # Remove ignored index (special tokens)
    predictions = [
        [entities_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    labels = [
        [entities_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return predictions, labels






In [42]:
results, results_strict,cr1,cr2 = compute_metrics_tr((annotated_sentences_first, dataset['test']['ner_tags']))

In [43]:
print(cr1)

              precision    recall  f1-score   support

   Condition       0.64      0.77      0.70      1105
      Device       0.24      0.30      0.27        23
        Drug       0.68      0.73      0.70       443
 Measurement       0.53      0.62      0.57       290
 Observation       0.30      0.18      0.23       166
      Person       0.76      0.84      0.80       135
   Procedure       0.46      0.49      0.48       313
    Temporal       0.48      0.58      0.52       297
       Value       0.65      0.70      0.68       351

   micro avg       0.60      0.67      0.63      3123
   macro avg       0.53      0.58      0.55      3123
weighted avg       0.59      0.67      0.62      3123



In [44]:
print(cr2)

              precision    recall  f1-score   support

   Condition       0.69      0.76      0.72      1104
      Device       0.29      0.30      0.30        23
        Drug       0.73      0.73      0.73       443
 Measurement       0.59      0.61      0.60       288
 Observation       0.40      0.17      0.24       166
      Person       0.76      0.84      0.80       135
   Procedure       0.53      0.49      0.51       311
    Temporal       0.58      0.57      0.58       295
       Value       0.70      0.72      0.71       345

   micro avg       0.66      0.66      0.66      3110
   macro avg       0.59      0.58      0.58      3110
weighted avg       0.65      0.66      0.65      3110



In [45]:
print(results)

{'Condition': {'precision': 0.6435568952524491, 'recall': 0.7728506787330317, 'f1': 0.7023026315789472, 'number': 1105}, 'Device': {'precision': 0.2413793103448276, 'recall': 0.30434782608695654, 'f1': 0.2692307692307692, 'number': 23}, 'Drug': {'precision': 0.6771488469601677, 'recall': 0.7291196388261851, 'f1': 0.7021739130434783, 'number': 443}, 'Measurement': {'precision': 0.5311572700296736, 'recall': 0.6172413793103448, 'f1': 0.5709728867623605, 'number': 290}, 'Observation': {'precision': 0.30303030303030304, 'recall': 0.18072289156626506, 'f1': 0.22641509433962267, 'number': 166}, 'Person': {'precision': 0.76, 'recall': 0.8444444444444444, 'f1': 0.8, 'number': 135}, 'Procedure': {'precision': 0.46107784431137727, 'recall': 0.49201277955271566, 'f1': 0.47604327666151475, 'number': 313}, 'Temporal': {'precision': 0.479108635097493, 'recall': 0.5791245791245792, 'f1': 0.5243902439024392, 'number': 297}, 'Value': {'precision': 0.65, 'recall': 0.7037037037037037, 'f1': 0.67578659370

In [46]:
print(results_strict)

{'Condition': {'precision': 0.6920529801324503, 'recall': 0.7572463768115942, 'f1': 0.7231833910034603, 'number': 1104}, 'Device': {'precision': 0.2916666666666667, 'recall': 0.30434782608695654, 'f1': 0.2978723404255319, 'number': 23}, 'Drug': {'precision': 0.7268623024830699, 'recall': 0.7268623024830699, 'f1': 0.7268623024830699, 'number': 443}, 'Measurement': {'precision': 0.5906040268456376, 'recall': 0.6111111111111112, 'f1': 0.6006825938566553, 'number': 288}, 'Observation': {'precision': 0.3972602739726027, 'recall': 0.1746987951807229, 'f1': 0.24267782426778242, 'number': 166}, 'Person': {'precision': 0.76, 'recall': 0.8444444444444444, 'f1': 0.8, 'number': 135}, 'Procedure': {'precision': 0.527972027972028, 'recall': 0.4855305466237942, 'f1': 0.5058626465661642, 'number': 311}, 'Temporal': {'precision': 0.5807560137457045, 'recall': 0.5728813559322034, 'f1': 0.5767918088737201, 'number': 295}, 'Value': {'precision': 0.7017045454545454, 'recall': 0.7159420289855073, 'f1': 0.70

In [47]:
results, results_strict = compute_metrics_tr((annotated_sentences_max, dataset['test']['ner_tags']))

ValueError: too many values to unpack (expected 2)

In [ ]:
print(results)

In [ ]:
print(results_strict)

In [ ]:
results["overall_accuracy"], results["overall_precision"], results["overall_f1"], results["overall_recall"]

In [ ]:
results_strict["overall_accuracy"], results_strict["overall_precision"], results_strict["overall_f1"], results_strict["overall_recall"]

In [ ]:
cr1 = classification_report([['B-Condition', 'I-Condition','I-Condition']], [['I-Condition', 'B-Condition','I-Condition']])
cr2 = classification_report([['B-Condition', 'I-Condition','I-Condition']], [['B-Condition', 'I-Condition','B-Condition']], mode='strict', scheme=IOB2)
print(cr1)
print(cr2)

In [ ]:
dataset_1 = load_dataset('JavierLopetegui/chia_v1')


In [ ]:
def get_entity_type(ent):
    """
    Get the entity type from the entity
    inputs:
        ent: str, the entity
    outputs:
        str: the entity type
    """
    if ent == 'O':
        return ent
    return ent.split('-')[1]
def get_recall(true_labels, pred_labels, mode='strict'):
    """
    Compute the recall for the model given a mode. The mode can be strict or relaxed.
    Strict mode implies that the entity type and the entity boundaries must match as well
    as the IOB tags for each subword. In relaxed mode, only the entity type and the entity
    boundaries must match.
    inputs:
        true_labels: list, the true labels
        pred_labels: list, the predicted labels
        mode: str, the mode to compute the recall
    outputs:
        recall: float, the recall of the model
        true_positive: int, corresponding to the true positive entities
        total_entities: int, total entities
        dict_metric_entity_type: dict, the metrics for each entity type
    """
    true_positive = 0
    total_entities = 0

    dict_metric_entity_type = {
        key: [0,0,0] for key in simple_ent
    }
    for i in range(len(true_labels)):
        if mode == 'strict':
            start_ent = 0
            while start_ent < len(true_labels[i]):
                if true_labels[i][start_ent] == 'O':
                    start_ent += 1
                    continue
                match_ent = True
                ent_type = get_entity_type(true_labels[i][start_ent])
                dict_metric_entity_type[ent_type][1] += 1
                if true_labels[i][start_ent] != pred_labels[i][start_ent]:
                    match_ent = False
                total_entities += 1
                start_ent += 1
                while start_ent < len(true_labels[i]) and true_labels[i][start_ent].startswith('I'):
                    if match_ent and true_labels[i][start_ent] != pred_labels[i][start_ent]:
                        match_ent = False
                    start_ent += 1
                if match_ent:
                    true_positive += 1
                    dict_metric_entity_type[ent_type][2] += 1

        elif mode == 'relaxed':
            start_ent = 0
            while start_ent < len(true_labels[i]):
                if true_labels[i][start_ent] == 'O':
                    start_ent += 1
                    continue
                match_ent = True
                ent_type = get_entity_type(true_labels[i][start_ent])
                dict_metric_entity_type[ent_type][1] += 1
                if get_entity_type(true_labels[i][start_ent]) != get_entity_type(pred_labels[i][start_ent]):
                    match_ent = False
                total_entities += 1
                start_ent += 1
                while start_ent < len(true_labels[i]) and true_labels[i][start_ent].startswith('I'):
                    if match_ent and get_entity_type(true_labels[i][start_ent]) != get_entity_type(pred_labels[i][start_ent]):
                        match_ent = False
                    start_ent += 1
                if match_ent:
                    true_positive += 1
                    dict_metric_entity_type[ent_type][2] += 1
        else:
            raise ValueError('Mode not recognized')
    for key in dict_metric_entity_type:
        if dict_metric_entity_type[key][1] != 0:
            dict_metric_entity_type[key][0] = dict_metric_entity_type[key][2]/dict_metric_entity_type[key][1]
    return (1.0*true_positive)/total_entities, true_positive, total_entities, dict_metric_entity_type


print(get_recall([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value']], [['I-Condition', 'B-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'relaxed'))
print(get_recall([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value']], [['I-Condition', 'B-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'strict'))
print(get_recall([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value']], [['B-Condition', 'I-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'strict'))
print(get_recall([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], [['B-Condition', 'I-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], 'strict'))

In [58]:
def get_precision(true_labels, pred_labels, mode='strict'):
    """
    Compute the precision for the model given a mode. The mode can be strict or relaxed.
    Strict mode implies that the entity type and the entity boundaries must match as well
    as the IOB tags for each subword. In relaxed mode, only the entity type and the entity
    boundaries must match.
    inputs:
        true_labels: list, the true labels
        pred_labels: list, the predicted labels
        mode: str, the mode to compute the precision
    outputs:
        precision: float, the precision of the model
        true_positive: int, corresponding to the true positive entities
        total_entities: int, total entities
    """
    return get_recall(pred_labels, true_labels, mode)

In [60]:
def get_metrics(true_labels, pred_labels, mode='strict'):
    """
    Compute recall, precision and f1-score for the model given a mode. The mode can be strict or relaxed.
    Strict mode implies that the entity type and the entity boundaries must match as well
    as the IOB tags for each subword. In relaxed mode, only the entity type and the entity
    boundaries must match.
    inputs:
        true_labels: list, the true labels
        pred_labels: list, the predicted labels
        mode: str, the mode to compute the metrics
    outputs:
        overall_recall: float, the recall of the model
        overall_precision: float, the precision of the model
        overall_f1: float, the f1-score of the model
        dict_metric_entity_type: dict, the metrics for each entity type
    """
    overall_recall, true_positive, total_true_entities, dict_recall = get_recall(true_labels, pred_labels, mode)
    overall_precision, true_positive_p, total_pred_entities, dict_precision = get_precision(true_labels, pred_labels, mode)
    overall_f1 = 2*overall_recall*overall_precision/(overall_recall + overall_precision)
    dict_metric_entity_type = { key: {'f1_score': 0, 'recall' : 0, 'precision' : 0} for key in simple_ent}
    for key in dict_recall:
        dict_metric_entity_type[key]['f1_score'] = 2*dict_recall[key][0]*dict_precision[key][0]/(dict_recall[key][0] + dict_precision[key][0]) if dict_recall[key][0] + dict_precision[key][0] != 0 else 0
        dict_metric_entity_type[key]['recall'] = dict_recall[key][0]
        dict_metric_entity_type[key]['precision'] = dict_precision[key][0]
    return {'overall_recall' : overall_recall, 'overall_precision' : overall_precision, 'overall_f1' : overall_f1, 'entities' : dict_metric_entity_type}

In [59]:
print(get_precision([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value']], [['I-Condition', 'B-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'relaxed'))
print(get_precision([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value']], [['I-Condition', 'B-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'strict'))
print(get_precision([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'I-Value']], [['B-Condition', 'I-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value']], 'strict'))
print(get_precision([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], [['B-Condition', 'I-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], 'strict'))

(1.0, 3, 3, {'Procedure': [0, 0, 0], 'Measurement': [0, 0, 0], 'Observation': [0, 0, 0], 'Person': [0, 0, 0], 'Device': [0, 0, 0], 'Drug': [0, 0, 0], 'Temporal': [0, 0, 0], 'Value': [1.0, 1, 1], 'Condition': [1.0, 2, 2]})
(0.0, 0, 3, {'Procedure': [0, 0, 0], 'Measurement': [0, 0, 0], 'Observation': [0, 0, 0], 'Person': [0, 0, 0], 'Device': [0, 0, 0], 'Drug': [0, 0, 0], 'Temporal': [0, 0, 0], 'Value': [0.0, 1, 0], 'Condition': [0.0, 2, 0]})
(1.0, 2, 2, {'Procedure': [0, 0, 0], 'Measurement': [0, 0, 0], 'Observation': [0, 0, 0], 'Person': [0, 0, 0], 'Device': [0, 0, 0], 'Drug': [0, 0, 0], 'Temporal': [0, 0, 0], 'Value': [1.0, 1, 1], 'Condition': [1.0, 1, 1]})
(0.75, 3, 4, {'Procedure': [0, 0, 0], 'Measurement': [0, 0, 0], 'Observation': [0, 0, 0], 'Person': [1.0, 1, 1], 'Device': [1.0, 1, 1], 'Drug': [0, 0, 0], 'Temporal': [0, 0, 0], 'Value': [0.0, 1, 0], 'Condition': [1.0, 1, 1]})


In [61]:
get_metrics([['B-Condition', 'I-Condition','I-Condition','O','B-Value', 'I-Value', 'B-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], [['B-Condition', 'I-Condition','I-Condition', 'O', 'B-Value', 'I-Value', 'I-Value'], ['B-Person', 'O', 'B-Device', 'I-Device']], 'strict')


{'overall_recall': 0.8,
 'overall_precision': 0.75,
 'overall_f1': 0.7741935483870969,
 'entities': {'Procedure': {'f1_score': 0, 'recall': 0, 'precision': 0},
  'Measurement': {'f1_score': 0, 'recall': 0, 'precision': 0},
  'Observation': {'f1_score': 0, 'recall': 0, 'precision': 0},
  'Person': {'f1_score': 1.0, 'recall': 1.0, 'precision': 1.0},
  'Device': {'f1_score': 1.0, 'recall': 1.0, 'precision': 1.0},
  'Drug': {'f1_score': 0, 'recall': 0, 'precision': 0},
  'Temporal': {'f1_score': 0, 'recall': 0, 'precision': 0},
  'Value': {'f1_score': 0.0, 'recall': 0.5, 'precision': 0.0},
  'Condition': {'f1_score': 1.0, 'recall': 1.0, 'precision': 1.0}}}

In [63]:
labels_predictions, true_labels = get_labels((annotated_sentences_first, dataset['test']['ner_tags']))

In [64]:
get_metrics(true_labels, labels_predictions)

{'overall_recall': 0.6853752405388069,
 'overall_precision': 0.6790805935408787,
 'overall_f1': 0.682213397485659,
 'entities': {'Procedure': {'f1_score': 0.537155088176446,
   'recall': 0.5016077170418006,
   'precision': 0.578125},
  'Measurement': {'f1_score': 0.6432601579819288,
   'recall': 0.6816608996539792,
   'precision': 0.608955223880597},
  'Observation': {'f1_score': 0.22288261515601784,
   'recall': 0.15060240963855423,
   'precision': 0.42857142857142855},
  'Person': {'f1_score': 0.8028169014084506,
   'recall': 0.8444444444444444,
   'precision': 0.7651006711409396},
  'Device': {'f1_score': 0.3137254901960784,
   'recall': 0.34782608695652173,
   'precision': 0.2857142857142857},
  'Drug': {'f1_score': 0.7485374406049823,
   'recall': 0.7584650112866818,
   'precision': 0.7388663967611336},
  'Temporal': {'f1_score': 0.6032632377386504,
   'recall': 0.6013513513513513,
   'precision': 0.6051873198847262},
  'Value': {'f1_score': 0.7597693657558285,
   'recall': 0.7542

In [65]:
get_metrics(true_labels, labels_predictions, 'relaxed')

{'overall_recall': 0.7350865939704939,
 'overall_precision': 0.723305208030259,
 'overall_f1': 0.7291483139752429,
 'entities': {'Procedure': {'f1_score': 0.5922165820642978,
   'recall': 0.5627009646302251,
   'precision': 0.625},
  'Measurement': {'f1_score': 0.7036808882253611,
   'recall': 0.7923875432525952,
   'precision': 0.6328358208955224},
  'Observation': {'f1_score': 0.22288261515601784,
   'recall': 0.15060240963855423,
   'precision': 0.42857142857142855},
  'Person': {'f1_score': 0.8101451938981805,
   'recall': 0.8444444444444444,
   'precision': 0.7785234899328859},
  'Device': {'f1_score': 0.3696098562628337,
   'recall': 0.43478260869565216,
   'precision': 0.32142857142857145},
  'Drug': {'f1_score': 0.7870745278336512,
   'recall': 0.7787810383747178,
   'precision': 0.7955465587044535},
  'Temporal': {'f1_score': 0.6399136115103536,
   'recall': 0.652027027027027,
   'precision': 0.6282420749279539},
  'Value': {'f1_score': 0.7911710193765795,
   'recall': 0.78,
 

In [13]:
added_pred = [['I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'B-age', 'I-age', 'I-age', 'I-age', 'I-age']]
added_true = [['B-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age', 'I-age']]

print(classification_report(added_true, added_pred))
print(classification_report(added_true, added_pred, mode='strict', scheme=IOB2))

              precision    recall  f1-score   support

         age       0.00      0.00      0.00         1

   micro avg       0.00      0.00      0.00         1
   macro avg       0.00      0.00      0.00         1
weighted avg       0.00      0.00      0.00         1

              precision    recall  f1-score   support

         age       0.00      0.00      0.00         1

   micro avg       0.00      0.00      0.00         1
   macro avg       0.00      0.00      0.00         1
weighted avg       0.00      0.00      0.00         1



In [6]:
help(classification_report)

Help on function classification_report in module seqeval.metrics.sequence_labeling:

classification_report(y_true, y_pred, digits=2, suffix=False, output_dict=False, mode=None, sample_weight=None, zero_division='warn', scheme=None)
    Build a text report showing the main classification metrics.
    
    Args:
        y_true : 2d array. Ground truth (correct) target values.
    
        y_pred : 2d array. Estimated targets as returned by a classifier.
    
        digits : int. Number of digits for formatting output floating point values.
    
        output_dict : bool(default=False). If True, return output as dict else str.
    
        mode : str, [None (default), `strict`].
            if ``None``, the score is compatible with conlleval.pl. Otherwise,
            the score is calculated strictly.
    
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights.
    
        zero_division : "warn", 0 or 1, default="warn"
            Sets the val